Install Necessary Libraries (Database Training Part)

In [ ]:
!pip install ultralytics
!nvidia-smi
pip install wandb
!pip install GPUtil

Set Training Configuration (Cloud Environment: Kaggle)

In [ ]:
# Build from YAML and transfer weights
model = YOLO('yolov8x.yaml').load('yolov8x.pt')  

# Training The Final Model
results = model.train(data="/kaggle/input/addbdvss/traffic_update.yaml",epochs=30, imgsz = 416, batch = 64 ,lr0=0.0001, dropout= 0.15, device = 0)

In [ ]:
from ultralytics import YOLO
import os
!yolo task=detect mode=train model=yolov8x.pt data="/kaggle/input/addbdvss/traffic_update.yaml" epochs=30 imgsz=640

Free Up Occupied GPU Storage

In [ ]:
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()                           


Install Necessary Libraries (Test Part)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install ultralytics
!pip install cvzone
!pip install supervision

Zone Based Collision Avoidance Alert  (Cloud Environment: Google Colab)

In [ ]:
# Import necessary libraries
import os
from ultralytics import YOLO
import cv2
from cvzone import putTextRect
import numpy as np
from cvzone import cornerRect
from cvzone import overlayPNG
import supervision as sv

# Define the directory to store videos
VIDEOS_DIR = os.path.join('.', 'videos')

# Specify the input video file and output video file paths
video_path = os.path.join(VIDEOS_DIR, r"/content/drive/MyDrive/YoLo_Thesis/BDV/Test Clips/Test6.mp4")
video_path_out = '{}_out.mp4'.format(video_path)

# Open the input video file for reading
cap = cv2.VideoCapture(video_path)

# Read the first frame of the video
ret, frame = cap.read()

# Get the height, width, and number of channels of the frame
H, W, _ = frame.shape

# Coordinates of the rectangular portion (ROI)
roi_x1, roi_y1, roi_x2, roi_y2 = 0, int(H/7), W, int((3*H)/5)  # Update these coordinates according to your ROI

# Create a VideoWriter object to save the output video
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

# Specify the path of the YOLO model
model_path1 = os.path.join('.', r"/content/drive/MyDrive/YoLo_Thesis/BDTS/runskg2/best(1).pt")
model_path2 = os.path.join('.', r"/content/drive/MyDrive/YoLo_Thesis/BDV/best.pt")

# Load the YOLO model
model1 = YOLO(model_path1)
model2 = YOLO(model_path2)

tracker = sv.ByteTrack()
trace_annotator = sv.TraceAnnotator()


# Set a threshold for object detection confidence
threshold = 0.5

Ap = np.array([14,15,22,23,24,25,26])
Ad = np.array([1,2,11,12,13,16,21])
Am= np.array([4,5,6,17,18,19])

# Process Graphics Icon

imgGraphicsPath = os.path.join('.', r"/content/drive/MyDrive/YoLo_Thesis/BDV/Test Clips/Capture3.png")
imgGraphics = cv2.imread(imgGraphicsPath, cv2.IMREAD_UNCHANGED)
height, width = imgGraphics.shape[:2]
imgGraphics = cv2.resize(imgGraphics, (int(height*0.7), int(width*0.7)))



# Process each frame of the input video
while ret:

    # Extract the ROI from the frame
    roi = frame[roi_y1:roi_y2, roi_x1:roi_x2]

    active_zone = False

    #Zone For LOS


    # Perform object detection on the ROI using the YOLO model
    #results1 = model1(roi)[0]
    results2 = model2(frame)[0]

    # Iterate through the detected objects in the ROI
    # for result1 in results1.boxes.data.tolist():
    #     x1, y1, x2, y2, score, class_id = result1

    #     # Check if the detection confidence is above the threshold
    #     if score > threshold:
    #         # Adjust the coordinates to the original frame
    #         x1 += roi_x1
    #         x2 += roi_x1
    #         y1 += roi_y1
    #         y2 += roi_y1

    #         class_name = results1.names[int(class_id)].upper()
    #         confidence = round(score * 100, 2)

    #         #cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 4)
    #         cornerRect(frame, (int(x1), int(y1), int(x2)-int(x1), int(y2)-int(y1)), l=6, rt=4, t=3, colorR=(0, 0, 255), colorC=(0, 255, 0))
    #         #cv2.putText(frame, f'{class_name} {confidence}%', (int(x1), int(y1 - 10)),
    #                     #cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 255), 3, cv2.LINE_AA)

    #         putTextRect(frame, f'{class_name} {confidence}%', [int(x1), int(y1) -15],
    #                     scale=1.5, thickness=2,colorR=(0,0,255), offset=7)

    #        #putTextRect(frame, f'{class_name} {confidence}%', [int(x1), int(y1) - 20],
    #                     #scale=1.5, thickness=2)

    #         if class_id in Ap:

    #          putTextRect(frame, f'{"PROHIBITORY"}', [int(x1) + 8, int(y2) +30],
    #                     scale=1.5, thickness=2,colorR=(0,0,255))


    #         elif class_id in Ad:

    #          putTextRect(frame, f'{"DANGER"}', [int(x1) + 8, int(y2) +30],
    #                     scale=1.5, thickness=2,colorR=(0,0,255))


    #         elif class_id in Am:

    #          putTextRect(frame, f'{"MANDATORY"}', [int(x1) + 8, int(y2) +30],
    #                     scale=1.5, thickness=2,colorR=(255,0,0))


    #         else:

    #          putTextRect(frame, f'{"OTHERS"}', [int(x1) + 8, int(y2) +30],
    #                     scale=1.5, thickness=2,colorR=(0,128,255))

    #         # Check if the current bounding box is within the specified ROI
    #         if ((x1 >= roi_x1) and (x2 <= roi_x2) and (y1 >= roi_y1) and (y2 <= roi_y2)):
    #                 active_zone = True

    # Check the condition here based on the active_zone variable



    for result2 in results2.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result2

        x1=int(x1)
        x2=int(x2)
        y1=int(y1)
        y2=int(y2)
        w=x2-x1
        cv2.rectangle(frame,  (int((W/2)-0.2*W), 0), (int((W/2)+0.2*W), H) , (0, 255, 0), 2)

        if score > threshold:

            class_name = results2.names[int(class_id)].upper()
            #confidence = round(score * 100, 2)
            #cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 4)
            #cornerRect(frame, (int(x1), int(y1), int(x2)-int(x1), int(y2)-int(y1)), l=8, rt=2, t=5, colorR=(153, 51, 255), colorC=(0, 255, 0))
            #cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                        #cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

            # putTextRect(frame, f'{class_name} {confidence}%', [int(x1), int(y1) -10],
            #                     scale=1, thickness=2, colorR=(153, 51, 255), offset=5)

            center = ((x1+x2)/2 , (y1+y2)/2)

            ratio = (w/W)
            cv2.putText(frame, f'{ratio:.2f}', (int(center[0]), int(center[1])-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            if ratio >0.29:

              if ((x1 >= int((W/2)-0.2*W)) or (x2 <= int((W/2)+0.2*W)) or (y1 >= 0) and (y2 <= H)):
                cv2.putText(frame, "CAUTION!", (int(center[0]), int(center[1])+20),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
                overlayPNG(frame, imgGraphics, (int(center[0]), int(y1-20)))

            p2x, p2y = 667, 483
            width = 60 #100

            p1 = 506, 720
            p2 = p2x, p2y
            p3 = p2x+width , p2[1]
            center = (p2[0]+p3[0])/2 , p2x
            p4 = p1[0] + (center[0]-p1[0])*2 , 720

            # Convert points to a numpy array
            pts = np.array([p1, p2, p3, p4], np.int32)
            pts = pts.reshape((-1, 1, 2))

            # Draw the polygon
            cv2.polylines(frame, [pts], isClosed=True, color=(0, 0, 255), thickness=4)
            #cv2.imshow("Image", img)

            zone1pts1 = 641 , 522
            zone1pts2 = (zone1pts1[0] + int(center[0]-zone1pts1[0])*2) , zone1pts1[1]
            cv2.line(frame, pt1=zone1pts1, pt2=zone1pts2, color=(255,0,0), thickness=4)

            ptsZone1 = np.array([zone1pts1, p2, p3, zone1pts2], np.int32)
            ptsZone1 = ptsZone1.reshape((-1, 1, 2))

            zone2pts1 = 593 , 592
            zone2pts2 = zone2pts1[0] + int(center[0]-zone2pts1[0])*2 , zone2pts1[1]
            cv2.line(frame, pt1=zone2pts1, pt2=zone2pts2, color=(255,0,0), thickness=4)

            ptsZone2 = np.array([zone2pts1, zone1pts1, zone1pts2, zone2pts2], np.int32)
            ptsZone2 = ptsZone2.reshape((-1, 1, 2))

            ptsZone3 = np.array([p1, zone2pts1, zone2pts2, p4], np.int32)
            ptsZone3 = ptsZone3.reshape((-1, 1, 2))



            # Set the mouse callback function
            #cv2.setMouseCallback('Image', get_coordinates)

            # Create a mask image the same size as the original image
            mask = np.zeros_like(frame)

            # Fill the polygon on the mask with a color (e.g., red with some transparency)
            cv2.fillPoly(mask, [ptsZone1], (0, 255, 0))
            cv2.fillPoly(mask, [ptsZone2], (0, 255, 255))
            cv2.fillPoly(mask, [ptsZone3], (0, 0, 255))

            # Combine the mask with the original image using addWeighted to keep transparency
            alpha = 0.4  # Transparency factor
            frame = cv2.addWeighted(frame, 1, mask, alpha, 0)

            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2

            # Define the corners of the bounding box
            corners = [(x1, y1), (x1, y2), (x2, y1), (x2, y2)]

            # Check if any corner of the bounding box is inside any zone
            in_green = any(cv2.pointPolygonTest(ptsZone1, corner, False) >= 0 for corner in corners)
            in_yellow = any(cv2.pointPolygonTest(ptsZone2, corner, False) >= 0 for corner in corners)
            in_red = any(cv2.pointPolygonTest(ptsZone3, corner, False) >= 0 for corner in corners)

            if in_red:
                  putTextRect(frame,"RED", [380,100], scale=2.5, thickness=3, colorR=(0, 0, 240), offset=8, border=2, colorB=(0,0,0))
            elif in_yellow:
                  putTextRect(frame,"YELLOW", [190,100], scale=2.5, thickness=3, colorR=(0, 128, 255), offset=8, border=2, colorB=(0,0,0))
            elif in_green:
                  putTextRect(frame,"GREEN", [20,100], scale=2.5, thickness=3, colorR=(0, 220, 0), offset=8, border=2, colorB=(0,0,0))

            # if cv2.pointPolygonTest(ptsZone1, (center_x, center_y), False) >= 0:
            #     cv2.putText(frame, "GREEN", (20,100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3, cv2.LINE_AA)
            # elif cv2.pointPolygonTest(ptsZone2, (center_x, center_y), False) >= 0:
            #     cv2.putText(frame, "YELLOW", (20,100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3, cv2.LINE_AA)
            # elif cv2.pointPolygonTest(ptsZone3, (center_x, center_y), False) >= 0:
            #     cv2.putText(frame, "RED", (20,100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3, cv2.LINE_AA)

    # if active_zone:
    #          cv2.rectangle(frame, (0, int(H/7)), (W, int((3*H)/5)), (255, 0, 0), 4)
    #          #cv2.putText(frame, "Active Virtual Zone", (int(W / 2) - 150, int(H / 2)), cv2.FONT_HERSHEY_COMPLEX, 1.3, (255, 0, 0), 3)
    #          putTextRect(frame, f'{"Active Virtual Zone"}', [int(W / 2) - 170, int(3*H /5)+10],
    #                     scale=2.5, thickness=2,colorR=(255,0,0))

    # else:
    #          cv2.rectangle(frame, (0, int(H/7)), (W, int((3*H)/5)), (0, 0, 255), 4)
    #          #cv2.putText(frame, "Inactive Virtual Zone", (int(W / 2) - 150, int(H / 2)), cv2.FONT_HERSHEY_COMPLEX, 1.3, (0, 255, 255), 3)
    #          putTextRect(frame, f'{"Inactive Virtual Zone"}', [int(W / 2) - 170, int(3*H /5)+10],
    #                     scale=2.5, thickness=2,colorR=(0,0,255))





    detections = sv.Detections.from_ultralytics(results2)
    #detections = detections.confidence > 0.5
    detections = tracker.update_with_detections(detections)
    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator(text_scale=0.5, text_thickness=2)



    labels = [
          #model2.model.names[class_id]
          #for class_id
          #in detections.class_id

        f"{tracker_id} {results2.names[class_id].upper()} {(confidence*100):.1f}%"
        for class_id, tracker_id, confidence
        in zip(detections.class_id, detections.tracker_id, detections.confidence)
      ]

    annotated_image = bounding_box_annotator.annotate(
          scene=frame, detections=detections)
    annotated_image = label_annotator.annotate(
          scene=annotated_image, detections=detections, labels=labels)
    annotated_image = trace_annotator.annotate(
          scene=annotated_image, detections=detections)


    # Write the frame to the output video file
    out.write(frame)

    # Read the next frame
    ret, frame = cap.read()

# Release the video capture and writer objects
cap.release()
out.release()

# Close all OpenCV windows
cv2.destroyAllWindows()
